In [77]:
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch
from torchvision.datasets import Food101
from trak import TRAKer


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [78]:
processor = AutoImageProcessor.from_pretrained("apple/mobilevit-small")
model = AutoModelForImageClassification.from_pretrained("apple/mobilevit-small")

In [81]:
def preprocess_image(image):
    image = transforms.functional.pil_to_tensor(image)
    # copying to avoid negative stride error of numpy array (seems necessary, sadly)
    processed_image = (processor.preprocess(image)["pixel_values"][0]).copy()
    return torch.from_numpy(processed_image)

dataset = Food101("data/food-101", transform=preprocess_image, download=True)

In [85]:
train_dl = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

for X, y in train_dl:
    break

X.shape, y.shape

(torch.Size([64, 3, 256, 256]), torch.Size([64]))

In [ ]:
# TODO finetune the model on food101 dataset -> use TRAK -> finetune again on the base model -> look at the results

In [84]:
traker = TRAKer(model, task="image_classification", train_set_size=len(dataset))

ERROR:TRAK:Could not use CudaProjector.
Reason: No module named 'fast_jl'
ERROR:TRAK:Defaulting to BasicProjector.
INFO:STORE:No existing model IDs in C:\Users\kamil\OneDrive\Pulpit\przedmioty\semestr 8\automating science\trak-for-automating-science\trak_results.
INFO:STORE:No existing TRAK scores in C:\Users\kamil\OneDrive\Pulpit\przedmioty\semestr 8\automating science\trak-for-automating-science\trak_results.
